In [4]:
# Using webcam recognize face
import torch
import pyttsx3
import cv2
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1
from torchvision import datasets
from tkinter import *
from PIL import Image, ImageTk
import os
from gtts import gTTS
import pyttsx3
import serial
import time


In [ ]:
# Define a video capture object
vid = cv2.VideoCapture(0)
 
# Declare the width and height in variables
width, height = 800, 600
  
# Set the width and height
vid.set(cv2.CAP_PROP_FRAME_WIDTH, width)
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
  
# Create a GUI app
app = Tk()

app.geometry("800x600")

  
# Bind the app with Escape keyboard to
# quit app whenever pressed
app.bind('<Escape>', lambda e: app.quit())


# Create a label and display it on app
label_widget = Label(app)
label_widget.pack()

  


def open_camera():
    # Capture the video frame by frame
    ret, frame = vid.read()
  
    # Convert image from one color space to other
    opencv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA) 
    # Capture the latest frame and transform to image
    captured_image = Image.fromarray(opencv_image)
    # Convert captured image to photoimage
    photo_image = ImageTk.PhotoImage(image=captured_image)
    # Displaying photoimage in the label
    label_widget.photo_image = photo_image
    # Configure image in the label
    label_widget.configure(image=photo_image)
    # Repeat the same process after every 10 seconds
    label_widget.after(10, open_camera)
    
    return vid.read()

In [ ]:
##############

def Vname(namel):
    engine = pyttsx3.init()
    rate = engine.getProperty('rate')   # getting details of current speaking rate
    engine.setProperty('rate', 125)     # setting up new voice rate
    voices = engine.getProperty('voices')       #getting details of current voice
    #engine.setProperty('voice', voices[0].id)  #changing index, changes voices. o for male
    engine.setProperty('voice', voices[1].id)   #changing index, changes voices. 1 for female
    volume = engine.getProperty('volume')   #getting to know current volume level (min=0 and max=1)
    engine.setProperty('volume',1.0)    # setting up volume level  between 0 and 1

    text = "Welcome" + namel
    engine.say(text)
    engine.runAndWait()
    


############### 

# initializing MTCNN and InceptionResnetV1 
mtcnn0 = MTCNN(image_size=240, margin=0, keep_all=False, min_face_size=40) # keep_all=False if multiple faces keep all false and detect one only
mtcnn = MTCNN(image_size=240, margin=0, keep_all=True, min_face_size=40) # keep_all=True to keep all faces in form of list
resnet = InceptionResnetV1(pretrained='vggface2').eval()  # using vggface2 pretrained model 

# Read data from folder

path = 'C:\\Users\\Dakhel\\'
dataset = datasets.ImageFolder(path + "FPhoto") # photos folder path 


# loading data.pt file
load_data = torch.load('data.pt') 
embedding_list = load_data[0] 
name_list = load_data[1] 
# cam = cv2.VideoCapture(0)     



def face():
    current_name = None
    counter_dakhel = 0
    counter_yaman = 0
    counter_mohammed = 0
    namev = 0 
    n = 8

    while True:
        ret, frame = open_camera()
#         ret, frame = cam.read()
        if not ret:
            print("fail to grab frame, try again")
            break

        img = Image.fromarray(frame)
        img_cropped_list, prob_list = mtcnn(img, return_prob=True) 

        if img_cropped_list is not None:
            boxes, _ = mtcnn.detect(img)

            for i, prob in enumerate(prob_list):
                if prob>0.90:
                    emb = resnet(img_cropped_list[i].unsqueeze(0)).detach() 

                    dist_list = [] # list of matched distances, minimum distance is used to identify the person

                    for idx, emb_db in enumerate(embedding_list):
                        dist = torch.dist(emb, emb_db).item()
                        dist_list.append(dist)

                    min_dist = min(dist_list) # get minumum dist value
                    min_dist_idx = dist_list.index(min_dist) # get minumum dist index
                    name = name_list[min_dist_idx] # get name corrosponding to minimum dist

                    box = boxes[i] 

                    original_frame = frame.copy() # storing copy of frame before drawing on it


                    if min_dist < 0.90:
                        namel = name
                        print(name)
                        if current_name != namel:
                            current_name = namel
                            counter_dakhel = 0
                            counter_yaman = 0
                            counter_mohammed = 0
                        if namel == "Dakhel":
                            counter_dakhel += 1
                            print(f"Name: {namel} - Count: {counter_dakhel}")
                        elif namel == "Yaman":
                            counter_yaman += 1
                            print(f"Name: {namel} - Count: {counter_yaman}")
                        elif namel == "Mohammed":
                            counter_mohammed += 1
                            print(f"Name: {namel} - Count: {counter_mohammed}")



        cv2.imshow("IMG", frame)


        k = cv2.waitKey(1)
        if k%256==27: # ESC
            print('Esc pressed, closing...')
            break

        if counter_dakhel == n:
            namev=1
            print("Welcome :" + namel + "_ Name Index : " + str(namev)) 
            Vname(namel)
            break

        if counter_mohammed == n:
            namev=2
            print("Welcome :" + namel + "_ Name Index : " + str(namev)) 
            Vname(namel)
            break

        if counter_yaman == n:
            namev=3
            print("Welcome :" + namel + "_ Name Index : " + str(namev)) 
            Vname(namel)
            break
    

cv2.destroyAllWindows()
# vid.release()




In [ ]:
open_camera()

def multiple_actions():
#     play_tts()
    face()
    

# Create a button to open the camera in GUI app
button1 = Button(app, text="Start", command=multiple_actions)
button1.pack()

app.mainloop()
